本notebook：
- 修改prompt，让llm将职责拆成两部分
- 根据岗位名称筛选特定大领域岗位信息
- 取llm返回的每个岗位信息的embedding，求余弦相似度，使相似性过强的岗位信息共用一个提取结果

In [1]:
import pandas as pd

data = pd.read_csv("D:\AI\AI-job-analysis\data_merge\merged_data_final.csv")
cv_condition = data["岗位名称"].str.contains("CV|视觉|图像|图形")

In [2]:
cv_data = data[cv_condition]
# 小批量测试数据
mini_cv_data = cv_data[2:6]
len(cv_data)

499

In [3]:
mini_cv_data.iloc[0]["岗位要求"]

'关于感知方向\n感知是无人驾驶中非常复杂和有趣的部分之一，你构建的是一个人工智能集大成的系统，不是一项按部就班就能完成的工作！感知软件工程师负责无人驾驶感知系统的设计和实现，应对无人驾驶中各种最有挑战的问题：\n1. 设计高效可靠的深度学习模型，在几十毫秒内精确检测和跟踪车周围200米之内所有的障碍物（人，车，非机动车辆，交通锥等），并对场景进行理解\n2. 如何设计一般性的模型和算法去处理各式各样的长尾情况和极端环境，如路面上的垃圾袋，洒水车的水花，前车掉下来的挡板 ，如大雨，大雪，雾霾，风沙等\n3. 如何保证感知模型和算法在极端的环境里的准确性和可靠性，如大雨，大雪，雾霾，风沙等\n4. 把模型优化到极致，让十几个到几十个模型在车上有限的计算资源上欢快的运行\n5. 如何搭建一个高效可靠的计算框架，支撑一个周期内接收几十个传感器的输入，做各种同步融合，并进行几十个深度学习模型的推理\n关于机器学习和算法方向\n这个方向的感知工程师负责设计并实现传感器标定，障碍物检测，分类，跟踪，和场景理解等各种模型和算法，对模型和算法进行评估和测试，并把模型和算法部署到车上。\n我们需要你：\n- 同时具有很强的算法和C++编程能力；或者在机器学习/深度学习，计算机视觉，模型优化等至少一个方向上有扎实的基础和丰富的经验。\n- 拥有强大的逻辑思维能力，算法能力，以及解决问题的能力。\n- 具有良好的沟通能力和跨组合作能力。\n- 具有强烈的自我驱动能力对无人驾驶真正的 believe 。\n加分项：\n- 丰富的C++开发经验，熟悉多线程，内存管理，泛型，及设计模式.\n- 丰富多传感器融合以及2d/3d跟踪的经验\n- 在顶会发表了文章'

In [8]:
import pickle

# 读取缓存文件，若缓存文件不存在或者为空（初次运行），则初始化一个空的字典
try:
    with open("data/all_keyword_dic.cache", "rb") as f:
        new_cache = pickle.load(f)
except:
    print("Empty cache file")
    new_cache = {}

try:
    with open("data/s2e.cache", "rb") as f:
        s2e = pickle.load(f)
except:
    print("Empty s2e file")
    s2e = {}

In [15]:
from test import *
# from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import json


for text in cv_data["岗位要求"]:
    prompt = f"""
请提取招聘信息中岗位所涉及的与招聘相关的关键词，分别为“语言需求”、“框架需求”、“业务领域”、“算法与开发”和“其他要求”。

关键词个数限制：
- 业务领域关键词不超过3个，更侧重表示金融、无人驾驶等与具体算法、技术无关的领域，不包括如“图像处理”等与算法、技术等相关领域。
- 框架需求关键词必须表示框架、库的要求，例如TensorFlow、PyTorch等。
- 算法与开发关键词侧重表示算法、模型、技术、工程、具体工作内容的要求，例如图像分类、目标检测、GAN、算法研究等。
- 其他要求中包括素质要求、学历要求、工作经验要求、优先条件等。
- 关键词应该具有一定的复杂性，避免过于简单的词汇，例如“算法研究”而不是“研究”或“算法”。
- 关键词中避免出现由或、和、顿号等连接的并列词汇，例如“有相关顶会论文或竞赛经验的优先”应当拆分为“顶会论文”“竞赛经验”
- 关键词中避免出现口语化表达和程度表达，例如“熟悉”“了解”“较好的”“良好的”等
- 若某一关键词类别不存在，则该关键词必须返回空列表，不要填入不完全属于该类别的内容

返回格式：
返回json，key为给定的6种关键词类别，value为具有一定复杂性的词汇的列表。
```
{text}
```
"""
    if text not in s2e:
        # get_embedding后，将文本与之前的文本进行相似度比较，如果相似度大于0.9，则将之前的文本的处理结果复制给当前文本
        s2e[text] = get_embedding(text, model="text-embedding-ada-002")
    for k, v in s2e.items():
        if k != text:
            if cosine_similarity(np.array(v), np.array(s2e[text])) > 0.988:
                print(k, "\n=========\n", text)
                new_cache[text] = new_cache[k]
                break
    # 如果文本已在缓存区，则跳过
    if text in new_cache:
        continue
    # 否则，进行ChatGPT_request
    jsons = ChatGPT_request(prompt)
    print(text)
    print("====================================")
    print(jsons)
    # 将ChatGPT_request的结果存入缓存区
    new_cache[text] = json.loads(jsons)

职位描述：
1. 进行多模态（文本与CV）方向算法研究和开发。
2. 负责图文的内容的标签挖掘，风险内容感知及热点分析等。
3. 基于大模型技术，提升风险识别效果，提升用户体验。
岗位要求：
1. 熟悉NLP、CV等相关技术，在NLP和CV方面有一定积累沉淀，有多模落地项目经验优先。
2. 负责多模态的算法框架构建，并基于多模态的特征表征，多模态的内容理解，提升下游风险识别任务效果。
3. 有扎实的编程基础，熟练掌握Tensorflow或Pytorch。
4. 责任心强，有良好的学习能力基团队合作精神，自我驱动力强，较强的沟通能力。 
 岗位职责：
1. 进行多模态（文本与CV）方向算法研究和开发。
2. 负责图文的内容的标签挖掘，风险内容感知及热点分析等。
3. 基于大模型技术，提升风险识别效果，提升用户体验。
岗位要求：
1. 熟悉NLP、CV等相关技术，在NLP和CV方面有一定积累沉淀，有多模落地项目经验优先。
2. 负责多模态的算法框架构建，并基于多模态的特征表征，多模态的内容理解，提升下游风险识别任务效果。
3. 有扎实的编程基础，熟练掌握Tensorflow或Pytorch。
4. 责任心强，有良好的学习能力基团队合作精神，自我驱动力强，较强的沟通能力。
职位描述：
1. 进行多模态（文本与CV）方向算法研究和开发。
2. 负责图文的内容的标签挖掘，风险内容感知及热点分析等。
3. 基于大模型技术，提升风险识别效果，提升用户体验。
岗位要求：
1. 熟悉NLP、CV等相关技术，在NLP和CV方面有一定积累沉淀，有多模落地项目经验优先。
2. 负责多模态的算法框架构建，并基于多模态的特征表征，多模态的内容理解，提升下游风险识别任务效果。
3. 有扎实的编程基础，熟练掌握Tensorflow或Pytorch。
4. 责任心强，有良好的学习能力基团队合作精神，自我驱动力强，较强的沟通能力。 
 岗位要求
职位描述：
1. 进行多模态（文本与CV）方向算法研究和开发。
2. 负责图文的内容的标签挖掘，风险内容感知及热点分析等。
3. 基于大模型技术，提升风险识别效果，提升用户体验。
岗位要求：
1. 熟悉NLP、CV等相关技术，在NLP和CV方面有一定积累沉淀，有多模落地项目经验优先。
2. 负责多模态的算法框架构建，并基于多模态的特征表征，多模态的内容理解，提

APIConnectionError: Connection error.

In [99]:
# import json
# new_cache = {k: json.loads(v) for k, v in new_cache.items()}

In [16]:
with open("data/all_keyword_dic.cache", "wb") as f:
    pickle.dump(new_cache, f)

In [17]:
with open("data/s2e.cache", "wb") as f:
    pickle.dump(s2e, f)

In [25]:
len(new_cache)

347

In [20]:
# 提取所有其他要求中的关键词
other_keywords = []
for k, v in new_cache.items():
    other_keywords.extend(v["其他要求"])
other_keywords = list(set(other_keywords))
len(other_keywords)

812

In [39]:
# 对其他要求中的关键词根据sequencematcher进行相似度比较，如果相似度大于0.9，则制作成字典
# 使用sequencematcher做初步筛选
from difflib import SequenceMatcher
from collections import defaultdict
k2kl = defaultdict(list)

for i in range(len(other_keywords)):
    for j in range(i + 1, len(other_keywords)):
        if "经验" in other_keywords[i] and "经验" in other_keywords[j]:
            continue
        if SequenceMatcher(None, other_keywords[i], other_keywords[j]).ratio() > 0.8:
            print(other_keywords[i], ":", other_keywords[j])
            k2kl[other_keywords[i]].append(other_keywords[j])

In [37]:
# 根据k2kl，将other_keywords中的关键词替换为相似度最高的关键词
for k, v in k2kl.items():
    for i in range(len(other_keywords)):
        if other_keywords[i] in v:
            other_keywords[i] = k

In [40]:
other_keywords = list(set(other_keywords))
len(other_keywords)

731

In [14]:
# common_keys = set(new_cache.keys()).intersection(s2e.keys())
# result_dict = {key: s2e[key] for key in common_keys}
# len(result_dict)
# s2e = result_dict